In [74]:
import json
import csv
import pandas as pd
import numpy as np
import time
import os
import uuid
import asyncio
from litellm import acompletion
from aiolimiter import AsyncLimiter


def generate_response_id():
    """Generate a unique ID for the response."""
    return str(uuid.uuid4())

In [75]:
from dotenv import load_dotenv

# Load environment variables from .env file
# LiteLLM automatically reads API keys from environment variables:
# - OPENAI_API_KEY for OpenAI models
# - GEMINI_API_KEY for Google Gemini models
# - ANTHROPIC_API_KEY for Anthropic models
load_dotenv()

True

In [76]:
async def create_response(model, user_input, response_schema, system_prompt):
    """Create a response using LiteLLM (supports multiple providers).
    
    Args:
        model: Model identifier (e.g., "gpt-4o", "gemini/gemini-2.0-flash", "anthropic/claude-3-5-sonnet-20241022")
        user_input: The user's input message
        response_schema: JSON schema for structured output
        system_prompt: System prompt for the LLM
    
    Returns:
        LiteLLM response object
    """
    response = await acompletion(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "statement_response",
                "strict": True,
                "schema": response_schema
            }
        }
    )
    return response

In [77]:
def construct_response_schema(response_options):
    return {
        "type": "object",
        "properties": {
            "input_statement": {
                "type": "string",
                "description": "The exact statement being responded to"
            },
            "response": {
                "type": "string",
                "description": "The response to the statement",
                "enum": response_options
            }
        },
        "required": ["input_statement", "response"],
        "additionalProperties": False
    }

In [78]:
def construct_generic_system_prompt(language, ai_definition, response_options):
    return f"""
    You are a primary or secondary school teacher. You have to respond to a survey statement in {language}.
    Please indicate your level of agreement with the statement about the use of artificial intelligence (AI) in education.
    In the survey, AI is defined as below in {language}:
    {ai_definition}

    Please respond with only one of the response options for each statement.
    Response options:
    {response_options}
    """

In [79]:
def verify_response_match_options(response, response_options):
    """Check if response is in the list of valid options. Returns bool."""
    return response in response_options

In [80]:
def construct_generic_user_input(statement):
    return f"Statement to respond to: {statement}"


In [81]:
def save_json_file(file_path, data):
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)


In [82]:
def save_response(folder, id, data):
    """Save full LLM response to responses/ folder.
    
    Args:
        response_json: The full response object as a dictionary
        
    Returns:
        str: The response ID
    """
    filepath = f"{folder}/{id}.json"
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    return id

In [83]:
def append_to_csv(filepath, row_data):
    """Append a single row to CSV, create with headers if file doesn't exist.
    
    Args:
        filepath: Path to the CSV file
        row_data: Dictionary containing row data
    """
    fieldnames = ['country', 'question_id', 'run_number', 'attempt', 'response_from_llm', 'response_score', 'model', 'llm_response_id']
    file_exists = os.path.exists(filepath)
    
    with open(filepath, 'a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(row_data)

In [84]:
async def process_statement(statement, run_number, response_options, response_options_obj, 
                            response_schema, system_prompt, csv_filepath, model, max_attempts,
                            country=None, response_folder=None):
    """Process a single statement with retry logic.
    
    Args:
        statement: Statement dict with 'id' and 'prompt'
        run_number: Current run iteration number
        response_options: List of valid response options
        response_options_obj: Dict mapping responses to scores
        response_schema: JSON schema for response validation
        system_prompt: System prompt for the LLM
        csv_filepath: Path to CSV file for logging
        model: The LLM model to use
        max_attempts: Maximum retry attempts for invalid responses
        country: Country name for country-based processing (optional)
        response_folder: Folder to save response JSON files (optional)
        
    Returns:
        bool: True if successful, False if all attempts failed
    """
    question_id = statement["id"]
    user_input = construct_generic_user_input(statement["prompt"])
    
    for attempt in range(1, max_attempts + 1):
        response = await create_response(model, user_input, response_schema, system_prompt)
        response_json = response.model_dump()
        # LiteLLM uses ChatCompletion format
        output = json.loads(response_json["choices"][0]["message"]["content"])
        
        # Generate unique ID and save full response to responses/ folder
        response_id = generate_response_id()
        if response_folder:
            save_response(response_folder, response_id, response_json)
        
        # Validate response
        if verify_response_match_options(output["response"], response_options):
            response_score = response_options_obj[output["response"]]
            row_data = {
                'question_id': question_id,
                'run_number': run_number,
                'attempt': attempt,
                'response_from_llm': output["response"],
                'response_score': response_score,
                'model': model,
                'llm_response_id': response_id
            }
            if country:
                row_data['country'] = country
            append_to_csv(csv_filepath, row_data)
            return True
        
        print(f"Invalid response '{output['response']}' for {question_id}, attempt {attempt}/{max_attempts}")
    
    print(f"FAILED: {question_id} after {max_attempts} attempts")
    return False

In [85]:
async def process_statement_with_rate_limit(rate_limiter, statement, run_number, 
                                             response_options, response_options_obj,
                                             response_schema, system_prompt, csv_filepath, 
                                             model, max_attempts, country=None, response_folder=None):
    """Wrapper that applies rate limiting before processing a statement.
    
    Args:
        rate_limiter: AsyncLimiter instance for rate limiting
        (all other args same as process_statement)
        
    Returns:
        bool: True if successful, False if all attempts failed
    """
    async with rate_limiter:
        return await process_statement(
            statement, run_number, response_options, response_options_obj,
            response_schema, system_prompt, csv_filepath, model, max_attempts,
            country, response_folder
        )

In [86]:
async def process_file(filepath):
    """Process all statements in a file for NUM_RUNS iterations.
    
    Args:
        filepath: Path to the questions JSON file
    """
    content = json.load(open(filepath, encoding='utf-8'))
    
    # Extract file name for CSV (e.g., "questions/usa_english.json" -> "generic/usa_english.csv")
    filename = os.path.basename(filepath).replace('.json', '.csv')
    csv_filepath = f"output/generic/{filename}"
    
    # Clear existing CSV for fresh run
    if os.path.exists(csv_filepath):
        os.remove(csv_filepath)
    
    # Setup from file content
    language = content["language"]
    ai_definition = content["questions"][0]["description"]
    response_options_obj = content["questions"][0]["responses"]
    response_options = list(response_options_obj.keys())
    response_schema = construct_response_schema(response_options)
    system_prompt = construct_generic_system_prompt(language, ai_definition, response_options)
    statements = content["questions"][0]["questions"]
    
    total_statements = len(statements)
    
    # Run NUM_RUNS times
    for run_number in range(1, NUM_RUNS + 1):
        print(f"[{filepath}] Run {run_number}/{NUM_RUNS}")
        for idx, statement in enumerate(statements, 1):
            print(f"  Processing statement {idx}/{total_statements}: {statement['id']}", end=" ")
            success = await process_statement(
                statement, run_number, response_options, response_options_obj,
                response_schema, system_prompt, csv_filepath
            )
            print("✓" if success else "✗")
    
    print(f"Completed: {filepath} -> {csv_filepath}")

In [46]:
# =============================================================================
# CONFIGURATION
# =============================================================================

MODEL = "gpt-4o"
MAX_ATTEMPTS = 3  # Maximum retry attempts for invalid responses
NUM_RUNS = 2      # Number of times to run each file

FILES = [
    # "questions/usa_english.json",
    "questions/alb_albanian.json",
]

for filepath in FILES:
    await process_file(filepath)


[questions/alb_albanian.json] Run 1/2
  Processing statement 1/10: TT4G35A ✓
  Processing statement 2/10: TT4G35B ✓
  Processing statement 3/10: TT4G35C ✓
  Processing statement 4/10: TT4G35D ✓
  Processing statement 5/10: TT4G35E ✓
  Processing statement 6/10: TT4G35F ✓
  Processing statement 7/10: TT4G35G ✓
  Processing statement 8/10: TT4G35H ✓
  Processing statement 9/10: TT4G35I ✓
  Processing statement 10/10: TT4G35J ✓
[questions/alb_albanian.json] Run 2/2
  Processing statement 1/10: TT4G35A ✓
  Processing statement 2/10: TT4G35B ✓
  Processing statement 3/10: TT4G35C ✓
  Processing statement 4/10: TT4G35D ✓
  Processing statement 5/10: TT4G35E ✓
  Processing statement 6/10: TT4G35F ✓
  Processing statement 7/10: TT4G35G ✓
  Processing statement 8/10: TT4G35H ✓
  Processing statement 9/10: TT4G35I ✓
  Processing statement 10/10: TT4G35J ✓
Completed: questions/alb_albanian.json -> output/generic/alb_albanian.csv


In [87]:
def construct_country_system_prompt(country, response_options):
    return f"""
    You are a primary or secondary school teacher from {country} responding to a survey about the use of artificial intelligence (AI) in education.
    Please indicate your level of agreement with the statement presented in the survey.
    In the survey, AI is defined as below:
    'Artificial intelligence' is the capacity for computers to perform tasks traditionally thought to involve human intelligence. This can include making predictions, suggesting decisions, or generating text.

    Please respond with only one of the response options for each statement.
    Response options:
    {response_options}
    """

In [88]:
async def process_all_countries(countries, statements, response_options, response_schema, 
                                 csv_filepath, response_folder, num_runs, models, max_attempts,
                                 requests_per_minute=60):
    """Process all statements for all countries and all models with rate limiting.
    
    Args:
        countries: List of country names
        statements: List of statement dicts with 'id' and 'prompt'
        response_options: List of valid response options
        response_schema: JSON schema for response validation
        csv_filepath: Path to CSV file for logging
        response_folder: Folder to save response JSON files
        num_runs: Number of times to run for each country
        models: List of model identifiers to use
        max_attempts: Maximum retry attempts for invalid responses
        requests_per_minute: Rate limit for API requests (default 60)
    """
    # Create rate limiter
    rate_limiter = AsyncLimiter(requests_per_minute, 60)  # X requests per 60 seconds
    
    response_options_obj = {opt: idx for idx, opt in enumerate(response_options, 1)}
    total_countries = len(countries)
    total_models = len(models)
    total_statements = len(statements)
    
    for model_idx, model in enumerate(models, 1):
        print(f"\n{'='*60}")
        print(f"[Model {model_idx}/{total_models}] {model}")
        print(f"{'='*60}")
        
        for country_idx, country in enumerate(countries, 1):
            print(f"\n[{country_idx}/{total_countries}] Processing country: {country}")
            system_prompt = construct_country_system_prompt(country, response_options)
            
            for run_number in range(1, num_runs + 1):
                print(f"  Run {run_number}/{num_runs} - Processing {total_statements} statements in parallel...")
                
                # Create tasks for all statements in this run
                tasks = [
                    process_statement_with_rate_limit(
                        rate_limiter, statement, run_number, response_options, response_options_obj,
                        response_schema, system_prompt, csv_filepath, model, max_attempts,
                        country, response_folder
                    )
                    for statement in statements
                ]
                
                # Execute all tasks in parallel (rate limiter controls throughput)
                results = await asyncio.gather(*tasks)
                
                success_count = sum(results)
                print(f"    Completed: {success_count}/{total_statements} successful")
    
    print(f"\nCompleted processing {total_models} models x {total_countries} countries -> {csv_filepath}")

In [89]:
def save_gen_config_details_to_file(output_folder, countries, statements, response_options, response_schema, num_runs, models, max_attempts):
    with open(f"{output_folder}/gen_config_details.txt", "w") as f:
        f.write(f"Countries: {countries}\n")
        f.write(f"Statements: {statements}\n")
        f.write(f"Response options: {response_options}\n")
        f.write(f"Response schema: {response_schema}\n")
        f.write(f"Num runs: {num_runs}\n")
        f.write(f"Models: {models}\n")
        

In [90]:
countries_file = "country_language_list.csv"

countries_df = pd.read_csv(countries_file)

unique_list_of_countries = countries_df["CNTRY_FULL"].unique().tolist()

print(len(unique_list_of_countries))


55


In [91]:
from datetime import datetime

# =============================================================================
# CONFIGURATION
# =============================================================================

# List of models to run (LiteLLM format: "provider/model" or just "model" for OpenAI)
# Examples:
#   - "gpt-4o", "gpt-4o-mini" (OpenAI)
#   - "gemini/gemini-2.0-flash", "gemini/gemini-1.5-pro" (Google Gemini)
#   - "anthropic/claude-3-5-sonnet-20241022" (Anthropic)
MODELS = [
    "gpt-5",
    "gpt-5.2",
    "gpt-5.1",
    "gpt-4o",
    "o3",
    "o4-mini",
    "gemini/gemini-2.0-flash",
    # "anthropic/claude-3-5-sonnet-20241022",
]

MAX_ATTEMPTS = 3  # Maximum retry attempts for invalid responses
NUM_RUNS = 1      

# Rate limiting - requests per minute
# Adjust based on your API tier:
#   - OpenAI: 500-10000 RPM depending on tier
#   - Gemini: 60-1000 RPM depending on tier  
#   - Anthropic: 50-4000 RPM depending on tier
REQUESTS_PER_MINUTE = 60

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

response_options = ["Strongly disagree", "Disagree", "Agree", "Strongly agree", "I don't know"]
response_schema = construct_response_schema(response_options)

list_of_statements = json.load(open("questions/usa_english.json", encoding='utf-8'))["questions"][0]["questions"]

output_folder = f"output/country_based_prompt/{timestamp}"
output_csv_file = f"{output_folder}/all.csv"
response_folder = f"{output_folder}/responses"

countries = unique_list_of_countries

# Create output folder if not exists
os.makedirs(output_folder, exist_ok=True)

# Create response folder if not exists
os.makedirs(response_folder, exist_ok=True)

# # Clear existing CSV for fresh run
# if os.path.exists(output_csv_file):
#     os.remove(output_csv_file)

save_gen_config_details_to_file(output_folder, countries, list_of_statements, response_options, response_schema, NUM_RUNS, MODELS, MAX_ATTEMPTS)

await process_all_countries(
    countries, list_of_statements, response_options, 
    response_schema, output_csv_file, response_folder, NUM_RUNS, MODELS, MAX_ATTEMPTS,
    REQUESTS_PER_MINUTE
)



[Model 1/7] gpt-5

[1/55] Processing country: United Arab Emirates
  Run 1/1 - Processing 10 statements in parallel...
    Completed: 10/10 successful

[2/55] Processing country: Australia
  Run 1/1 - Processing 10 statements in parallel...
    Completed: 10/10 successful

[3/55] Processing country: Belgium
  Run 1/1 - Processing 10 statements in parallel...
    Completed: 10/10 successful

[4/55] Processing country: Flemish Comm. (Belgium)
  Run 1/1 - Processing 10 statements in parallel...
    Completed: 10/10 successful

[5/55] Processing country: French Comm. (Belgium)
  Run 1/1 - Processing 10 statements in parallel...
    Completed: 10/10 successful

[6/55] Processing country: Brazil
  Run 1/1 - Processing 10 statements in parallel...
    Completed: 10/10 successful

[7/55] Processing country: Spain
  Run 1/1 - Processing 10 statements in parallel...
    Completed: 10/10 successful

[8/55] Processing country: France
  Run 1/1 - Processing 10 statements in parallel...
    Complet